<a href="https://colab.research.google.com/github/svanlinden/test/blob/master/Voorbeeldcode_Metaheuristieken.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install inspyred

     |████████████████████████████████| 92kB 3.8MB/s 


# Genetische algoritmen

## 1. Probleem voorstellen als een chromosoom

$(\bar a \vee c) \land (\bar a \vee c \vee \bar e) \land (\bar b \vee c \vee d \vee \bar e) \land (a \vee \bar b \vee c) \land (\bar e \vee f)$


In [ ]:
import itertools
import pandas as pd

def expand_grid(data_dict):
    """Create a dataframe from every combination of given values."""
    rows = itertools.product(*data_dict.values())
    return pd.DataFrame.from_records(rows, columns=data_dict.keys())

dictionary = {'a': [0, 1], 'b': [0, 1], 'c': [0, 1], 'd': [0, 1], 'e': [0, 1], 'f' : [0 , 1]}

table = expand_grid(dictionary)
table

,a,b,c,d,e,f
0,0,0,0,0,0,0
1,0,0,0,0,0,1
2,0,0,0,0,1,0
3,0,0,0,0,1,1
4,0,0,0,1,0,0
...,...,...,...,...,...,...
59,1,1,1,0,1,1
60,1,1,1,1,0,0
61,1,1,1,1,0,1
62,1,1,1,1,1,0


## 2. Fitnessformule schrijven (objective function)

In [ ]:
def formule(variables : [bool]):
    a, b, c, d, e, f = variables
    p1 = not a or c
    p2 = not a or c or not e
    p3 = not b or c or d or not e
    p4 = a or not b or c
    p5 = not e or f
    return bool(p1 and p2 and p3 and p4 and p5)


In [ ]:
formule([True, True, True, True, True, True])

True

In [ ]:
table.apply(formule, axis=1)

0      True
1      True
2     False
3      True
4      True
      ...  
59     True
60     True
61     True
62    False
63     True
Length: 64, dtype: bool

In [ ]:
def formule_fitness(variabeles : [bool]):
    a, b, c, d, e, f = variabeles
    p1 = not a or c
    p2 = not a or c or not e
    p3 = not b or c or d or not e
    p4 = a or not b or c
    p5 = not e or f
    return sum([p1, p2, p3, p4, p5])

In [ ]:
formule_fitness([False, True , False, False, True, False])

2

In [ ]:
pd.concat([table, pd.Series(table.apply(formule_fitness, axis=1),name='fitness')],axis=1)

,a,b,c,d,e,f,fitness
0,0,0,0,0,0,0,5
1,0,0,0,0,0,1,5
2,0,0,0,0,1,0,4
3,0,0,0,0,1,1,5
4,0,0,0,1,0,0,5
...,...,...,...,...,...,...,...
59,1,1,1,0,1,1,5
60,1,1,1,1,0,0,5
61,1,1,1,1,0,1,5
62,1,1,1,1,1,0,4


## Stappen 3 t.e.m. 10: Inspyred library gebruiken

Minimaal nodig:
* generatorfunctie
* evaluatorfunctie

### Generatorfunctie schrijven

In [ ]:
import numpy as np

def generate(random = None, args =  None) -> []:
    return np.random.randint(low=0, high=2, size=6)

In [ ]:
candidate  = generate()
print('Candidate solution:', candidate)

print('Candidate fitness:', formule_fitness(candidate))

Candidate solution: [0 0 1 1 0 1]
Candidate fitness: 5


### Evaluatorfunctie schrijven


In [ ]:
def evaluate(candidates, args = {}):
    fitness = []
    for candidate in candidates:
        fitness.append(formule_fitness(candidate))
    return fitness

### GA algoritme laten zoeken naar (sub)optimale oplossing

In [ ]:
from random import Random
import inspyred
from inspyred.ec import Individual

rand = Random()
ga = inspyred.ec.GA(rand)

population: [Individual] = ga.evolve(
    generator=generate,
    evaluator=evaluate,
    pop_size=100,
    maximize=True,
    bounder=inspyred.ec.Bounder(0, 1),
    max_evaluations=1000,
    mutation_rate=0.25)

# Sort and print the best individual, who will be at index 0.
population.sort(reverse=True)
for solution in population:
    print('{} -> {}'.format(solution.candidate, formule_fitness(solution.candidate)))


[0 0 1 1 0 0] -> 5
[1 1 1 0 0 1] -> 5
[0 1 1 1 0 0] -> 5
[1 1 1 0 1 1] -> 5
[1 0 1 1 1 1] -> 5
[0 0 0 0 1 1] -> 5
[0 1 1 1 0 0] -> 5
[0 0 1 0 1 1] -> 5
[0 0 1 1 0 1] -> 5
[0 0 0 0 0 0] -> 5
[1 1 1 0 1 1] -> 5
[1 0 1 0 0 0] -> 5
[1 0 1 1 0 0] -> 5
[1 1 1 1 0 0] -> 5
[1 1 1 1 0 1] -> 5
[0 0 1 1 1 1] -> 5
[0 0 1 1 0 1] -> 5
[1 0 1 0 0 1] -> 5
[1 1 1 0 0 1] -> 5
[1 0 1 1 0 1] -> 5
[1 1 1 0 0 1] -> 5
[1 1 1 0 0 0] -> 5
[0 0 0 1 0 0] -> 5
[0 0 0 1 0 1] -> 5
[0 0 0 0 0 0] -> 5
[1 0 1 1 0 0] -> 5
[0 0 0 1 0 1] -> 5
[0 0 1 1 0 1] -> 5
[0 0 1 1 0 1] -> 5
[0 1 1 0 0 0] -> 5
[0 0 0 0 0 1] -> 5
[0 1 1 0 1 1] -> 5
[0 1 1 0 1 1] -> 5
[0 0 1 0 0 0] -> 5
[1 0 1 1 0 1] -> 5
[1 1 1 0 0 0] -> 5
[1 1 1 0 0 1] -> 5
[1 1 1 1 1 1] -> 5
[0 1 1 0 0 1] -> 5
[0 0 1 0 0 0] -> 5
[1 0 1 0 0 1] -> 5
[1 1 1 0 1 1] -> 5
[1 1 1 0 0 0] -> 5
[1 1 1 1 0 1] -> 5
[1 1 0 1 0 1] -> 4
[1 1 0 1 0 0] -> 4
[1 1 0 0 0 1] -> 4
[0 1 0 1 0 1] -> 4
[1 1 0 1 0 0] -> 4
[0 0 0 1 1 0] -> 4
[0 0 0 0 1 0] -> 4
[1 0 1 0 1 0] -> 4
[1 0 1 1 1 0

### SA Algoritme laten zoeken naar (sub)optimale oplossing

In [ ]:
sa = inspyred.ec.SA(rand)
sa.terminator = inspyred.ec.terminators.evaluation_termination

population: [Individual] = sa.evolve(
    generator = generate,
    evaluator = evaluate,
    maximize = True,
    bounder = inspyred.ec.Bounder(0, 1),
    temperature = 1000,
    cooling_rate = 0.5,
    max_evaluations = 1000,
    mutation_rate = 0.25)

# Sort and print the best individual, who will be at index 0.
population.sort(reverse=True)
for solution in population:
    print('{} -> {}'.format(solution.candidate,formule_fitness(solution.candidate)))


[0 0 0 0 0 0] -> 5


In [1]:
print('opvulling om te kunnen afdrukken \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n')

opvulling om te kunnen afdrukken 
















